### Dependecies and necessary tools

In [1]:
#Importing dependencies
import pandas as pd
from sodapy import Socrata
import scipy.stats as st
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pymongo

# Importing data for API call
from config import app_token
from config import username
from config import password

### EXTRACT 

In [2]:
# Getting the data from API NY Open Data for the first database: 
client = Socrata("data.cityofnewyork.us",
                  app_token, username,
                  password)
# Database returned as JSON from API / converted to Python list of dictionaries by sodapy.
results = client.get("gaq9-z3hz", limit=2832)

# Convert to pandas DataFrame
Recycling_Div_Captures = pd.DataFrame.from_records(results)
Recycling_Div_Captures

,_zone,district,fiscal_month_number,fiscal_year,month_name,diversion_rate_total_total_recycling_total_waste_,capture_rate_paper_total_paper_max_paper_,capture_rate_mgp_total_mgp_max_mgp_,capture_rate_total_total_recycling_leaves_recycling_max_paper_max_mgp_x100
0,Brooklyn North,BKN01,10,2019,April,14.6870926033314,44.9091597758182,43.0340618265488,44.1467643388238
1,Brooklyn North,BKN02,10,2019,April,19.9501814568266,34.1940201111571,57.9470313498761,41.2136999269730
2,Brooklyn North,BKN03,10,2019,April,12.1641613296565,33.5215567195417,44.9197311047560,38.1559365633997
3,Brooklyn North,BKN04,10,2019,April,15.5418031271589,35.2113607057133,68.5112600518364,48.7507554470644
4,Brooklyn North,BKN05,10,2019,April,10.0518454148520,22.2654301541493,45.0517913754661,31.5301290076712
...,...,...,...,...,...,...,...,...,...
2827,Queens West,QW06,3,2016,September,20.1214735183925,30.4196893217642,67.9561359185218,39.0333079418224
2828,Queens West,QW09,3,2016,September,17.4348618222880,41.1065470054454,79.6950721141275,54.2727362237205
2829,Staten Island,SI01,3,2016,September,18.6838350777916,39.5199429139324,71.7164916919349,49.6526689393673
2830,Staten Island,SI02,3,2016,September,19.0168187778538,44.4579262409406,74.9601995452711,54.0574389302287


In [3]:
# creating path for the citywide subsort 
data_path = "Resources/Citywide_Subsort.csv"
# Read the data into df
cws_df = pd.read_csv(data_path)
#Preview data
cws_df.head()

,Material,Aggregate Percent,Refuse Percent,MGP Percent,Paper Percent,Organic Percent,Material Group,DSNY Diversion Summary Category,Location
0,Newspaper,0.019,0.011,0.005,0.098,0.000,Paper,Designated Paper,Citywide
1,Plain OCC/Kraft Paper,0.056,0.012,0.010,0.464,0.002,Paper,Designated Paper,Citywide
2,Paper Bags: Kraft Grocery,0.002,0.001,0.000,0.008,0.000,Paper,Designated Paper,Citywide
3,Paper Bags: Non-food Retail,0.002,0.002,0.000,0.008,0.000,Paper,Designated Paper,Citywide
4,Paper Bags: Fast Food Bags,0.001,0.002,0.000,0.001,0.000,Paper,Designated Paper,Citywide


In [4]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\da34s\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [5]:
url = 'https://data.cityofnewyork.us/City-Government/DSNY-Waste-Characterization-Mainsort/k3ks-jzek'
browser.visit(url)

In [6]:
df_list = []
for x in range(1,41):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    table = soup.find_all('div', class_='socrata-table frozen-columns')
    df_list.append(pd.read_html(str(table))[0])
    button = browser.find_by_xpath('//*[@id="app"]/div/div[2]/section[4]/div[2]/div/div/div[4]/div[3]/span[1]/button[2]').click()

ImportError: html5lib not found, please install it

In [ ]:
df_combined = pd.concat(df_list)

In [ ]:
df_combined.reset_index(drop=True, inplace=True)

In [ ]:
cwm_df = df_combined
cwm_df.head()

### TRANSFORM

In [ ]:
# Analizing material groups
unique_mat_subsort = cws_df["Material Group"].unique()
print(f'Material groups (subsort): {unique_mat_subsort}')
unique_mat_mainsort = cwm_df["Material Group"].unique()
print(f'Material groups (mainsort): {unique_mat_mainsort}')

In the general database (Recycling_Div_Captures) is possible to see that there is just data for paper and MGP, so from this dataset information just for those materials will be taken. 
The materials will be group in the next collections for a non-relational dabatase:
- Paper
- MGP (Metal, Glass, and Plastic)
- Organic
- E- Waste
- Special Waste
- Others

In [ ]:
unique_loc_subsort = cws_df["Location"].unique()
print(f"Locations register in the Subsort dataset: {unique_loc_subsort}")
print("----------------------------------------------------------------")
unique_loc_mainsort = cwm_df["Location"].unique()
print(f"Locations register in the Mainsort dataset: {unique_loc_mainsort}")
print("----------------------------------------------------------------")
uniqie_loc_general = Recycling_Div_Captures["_zone"].unique()
print(f"Locations register in the general dataset: {uniqie_loc_general}")

Analaizing the locations in all the databases, we decide to summarize and filtering them and their data just for the 5 boroughs.
- Manhattan
- Queens (which includes 'Queens East' and 'Queens West')
- Brooklyn (which includes 'Brooklyn North' and 'Brooklyn South')
- Bronx
- Staten Island


Each material collection will have some next fields per location (materials like paper and MGP will have average capture rates taken from the general dataset, but the others will not because there is not information):
- Average aggregate percent subsort
- Average refuse percent subsort
- Average aggregate percent mainsort
- Average refuse percent mainsort
- Average capture rate

In [ ]:
# Changing zone name in the general database
Recycling_Div_Captures["_zone"]= Recycling_Div_Captures["_zone"].str.replace(r'Brooklyn North', r'Brooklyn')
Recycling_Div_Captures["_zone"]= Recycling_Div_Captures["_zone"].str.replace(r'Brooklyn South', r'Brooklyn')
Recycling_Div_Captures["_zone"]= Recycling_Div_Captures["_zone"].str.replace(r'Queens East', r'Queens')
Recycling_Div_Captures["_zone"]= Recycling_Div_Captures["_zone"].str.replace(r'Queens West', r'Queens')
Recycling_Div_Captures

In [ ]:
# Subsort percentages format 
cols = ["Aggregate Percent", "Refuse Percent", "MGP Percent", "Paper Percent", "Organic Percent"]
cws_df[cols] = cws_df[cols] *100
cws_df.head()

In [ ]:
# # Mainsort percentages format 
cols = ["Aggregate Percent", "Refuse Percent", "MGP Percent", "Paper Percent", "Organic Percent"]
cwm_df["Aggregate Percent"]= cwm_df["Aggregate Percent"].str.replace(r'%', r'')
cwm_df["Refuse Percent"]= cwm_df["Refuse Percent"].str.replace(r'%', r'')
cwm_df["MGP Percent"]= cwm_df["MGP Percent"].str.replace(r'%', r'')
cwm_df["Paper Percent"]= cwm_df["Paper Percent"].str.replace(r'%', r'')
cwm_df["Organic Percent"]= cwm_df["Organic Percent"].str.replace(r'%', r'')

cwm_df[cols]= cwm_df[cols].apply(pd.to_numeric, errors='coerce')
print(cwm_df.dtypes)
cwm_df.head()

In [ ]:
# First filter: Recycling_Div_Captures data should be filter just for 2017 year
Recycling_Div_Captures["fiscal_year"] = Recycling_Div_Captures["fiscal_year"].apply(pd.to_numeric, errors='coerce')
Recycling_Div_Captures = Recycling_Div_Captures.loc[Recycling_Div_Captures["fiscal_year"]==2017,:]

In [ ]:
# Paper collection
# Materials paper types register in subsort
paper_material_sub = cws_df.loc[cws_df["Material Group"]=="Paper",["Material"]]
paper_material_sub = paper_material_sub["Material"].unique()
set_sub = set(paper_material_sub)

In [ ]:
# Materials paper types register in mainsort
paper_material_main = cwm_df.loc[cws_df["Material Group"]=="Paper",["Material"]]
paper_material_main = paper_material_main["Material"].unique()
set_main = set(paper_material_main)

In [ ]:
# Mergin material without duplication 
no_duplicates = list(set_main-set_sub)
paper_material_sub = list(set_sub)
paper_material_final = paper_material_sub + no_duplicates
paper_material_final

In [ ]:
# Filter just papper material group subsort
paper_sub_df = cws_df.loc[cws_df["Material Group"]=="Paper",:]
# Filter just papper material group mainsort
paper_main_df = cwm_df.loc[cws_df["Material Group"]=="Paper",:]

# Group and calculate the mean of aggregate and refuse percentage per location
agg_subsort = paper_sub_df.groupby("Location")["Aggregate Percent"].mean()
refuse_subsort = paper_sub_df.groupby("Location")["Refuse Percent"].mean()
agg_mainsort = paper_main_df.groupby("Location")["Aggregate Percent"].mean()
refuse_mainsort = paper_main_df.groupby("Location")["Refuse Percent"].mean()

# Collection paper will have Average capture rate taken from Recycling_Div_Captures DataFrame
avg_cap_rate = Recycling_Div_Captures.groupby("_zone")["capture_rate_paper_total_paper_max_paper_"].mean()

### LOAD
Create a non-relational dabase to load in MongoDB

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
materials_db = client["MaterialsDB"]
materials_coll = materials_db["materials"]


In [ ]:
# Insert paper document in the database
materials_db.materials_coll.insert_one(
    {
        'material_group': "Paper",
        'year': 2017,
        'materials_list': paper_material_final,
        'Bronx':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort.Bronx,
                'Refuse Percent': refuse_subsort.Bronx,
            }
#             'Mainsort':
#             {
#                 'Aggregate Percent': agg_mainsort.Bronx,
#                 'Refuse Percent': refuse_mainsort.Bronx,
#             }
            'Capture rate': avg_cap_rate.Bronx,
        }
        'Manhattan':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort.Manhattan,
                'Refuse Percent': refuse_subsort.Manhattan,
            }
            'Mainsort':
            {
                'Aggregate Percent': agg_mainsort.Manhattan,
                'Refuse Percent': refuse_mainsort.Manhattan,
            }
            'Capture rate': avg_cap_rate.Manhattan,
        }
        'Queens':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort.Queens,
                'Refuse Percent': refuse_subsort.Queens,
            }
#             'Mainsort':
#             {
#                 'Aggregate Percent': agg_mainsort.Queens,
#                 'Refuse Percent': refuse_mainsort.Queens,
#             }
            'Capture rate': avg_cap_rate.Queens,
        }
        'Brooklyn':
        {
            'Subsort':
            {
                'Aggregate Percent': agg_subsort.Brooklyn,
                'Refuse Percent': refuse_subsort.Brooklyn,
            }
#             'Mainsort':
#             {
#                 'Aggregate Percent': agg_mainsort.Brooklyn,
#                 'Refuse Percent': refuse_mainsort.Brooklyn,
#             }
            'Capture rate': avg_cap_rate.Brooklyn,
        }      

    }
)